In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

2024-01-29 00:42:26.036390: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/hugo/tensorflow-metal-test/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
#Preprocessing --- Goal to make the data compatible for CNN input
# In tensorflow, we can achieve the same using ImageGenerators

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1.0/255.)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1.0/255.)

In [3]:
#Pass the images through the generator
trainImageData = train_generator.flow_from_directory("../data/training",
                                                     batch_size=32, #how much images to give per iteration in an epoch
                                                     class_mode="binary", #Incase of multi-class classification, "categorical"
                                                     target_size=(64,64) #Ensures all images are of same size (resizing)
                                                     ) 


testImageData = train_generator.flow_from_directory("../data/testing",
                                                     batch_size=32, #how much images to give per iteration in an epoch
                                                     class_mode="binary", #Incase of multi-class classification, "categorical"
                                                     target_size=(64,64) #Ensures all images are of same size (resizing)
                                                     ) 

Found 2392 images belonging to 2 classes.
Found 597 images belonging to 2 classes.


In [4]:
#Given dataset is a balanced dataset

trainImageData.class_indices

{'flip': 0, 'notflip': 1}

In [5]:
trainImageData.filenames

['flip/0001_000000010.jpg',
 'flip/0001_000000011.jpg',
 'flip/0001_000000012.jpg',
 'flip/0001_000000013.jpg',
 'flip/0001_000000014.jpg',
 'flip/0001_000000015.jpg',
 'flip/0001_000000016.jpg',
 'flip/0001_000000017.jpg',
 'flip/0001_000000018.jpg',
 'flip/0001_000000019.jpg',
 'flip/0001_000000021.jpg',
 'flip/0001_000000022.jpg',
 'flip/0001_000000023.jpg',
 'flip/0001_000000024.jpg',
 'flip/0001_000000025.jpg',
 'flip/0002_000000009.jpg',
 'flip/0002_000000010.jpg',
 'flip/0002_000000011.jpg',
 'flip/0002_000000014.jpg',
 'flip/0002_000000016.jpg',
 'flip/0002_000000018.jpg',
 'flip/0002_000000019.jpg',
 'flip/0002_000000020.jpg',
 'flip/0002_000000021.jpg',
 'flip/0003_000000001.jpg',
 'flip/0003_000000002.jpg',
 'flip/0003_000000003.jpg',
 'flip/0003_000000004.jpg',
 'flip/0003_000000006.jpg',
 'flip/0003_000000007.jpg',
 'flip/0003_000000009.jpg',
 'flip/0003_000000010.jpg',
 'flip/0003_000000011.jpg',
 'flip/0003_000000012.jpg',
 'flip/0003_000000013.jpg',
 'flip/0003_00000001

In [6]:
trainImageData.image_shape

(64, 64, 3)

# Loading ResNet from keras to perform transfer learning

In [27]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [28]:
# Importing the ResNet model but discarding the last 1000 neuron layer for adjustment to our dataset
base_model = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape=(64, 64, 3))

In [29]:
# Freeze the base_model so we keep the pre-trained structure
base_model.trainable = False

In [30]:
# Adjusting the model to our dataset

model = tf.keras.models.Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152 (Functional)      (None, 2, 2, 2048)        58370944  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_4 (Dense)             (None, 1024)              8389632   
                                                                 
 dense_5 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 66,761,601
Trainable params: 8,390,657
Non-trainable params: 58,370,944
_________________________________________________________________


In [32]:
#Compile

model.compile(optimizer="adam",
              loss="binary_crossentropy", #For multi-class classification: categorical_crossentropy | sparse_categorical_crossentropy
              metrics=[tfa.metrics.F1Score(num_classes=1, threshold=0.5)])

In [33]:
len(trainImageData.filenames) // trainImageData.batch_size

74

In [34]:
# Fit the model
model.fit(trainImageData,
          validation_data=testImageData,
          epochs=20,
          steps_per_epoch = len(trainImageData.filenames) // trainImageData.batch_size,
          validation_steps= len(testImageData.filenames) // testImageData.batch_size)

Epoch 1/20


2024-01-29 01:14:09.554411: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-01-29 01:14:12.205276: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-01-29 01:14:12.469259: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


74/74 [==============================] - ETA: 0s - loss: 1.0006 - f1_score: 0.5918

2024-01-29 01:14:46.096896: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-01-29 01:14:46.764162: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


74/74 [==============================] - 47s 568ms/step - loss: 1.0006 - f1_score: 0.5918 - val_loss: 0.6400 - val_f1_score: 0.7051
Epoch 2/20
74/74 [==============================] - 38s 519ms/step - loss: 0.6253 - f1_score: 0.6708 - val_loss: 0.6138 - val_f1_score: 0.6754
Epoch 3/20
74/74 [==============================] - 38s 518ms/step - loss: 0.6108 - f1_score: 0.6878 - val_loss: 0.6407 - val_f1_score: 0.4393
Epoch 4/20
74/74 [==============================] - 38s 516ms/step - loss: 0.6023 - f1_score: 0.6875 - val_loss: 0.6145 - val_f1_score: 0.6341
Epoch 5/20
74/74 [==============================] - 38s 515ms/step - loss: 0.5737 - f1_score: 0.7231 - val_loss: 0.6034 - val_f1_score: 0.6894
Epoch 6/20
74/74 [==============================] - 38s 513ms/step - loss: 0.6351 - f1_score: 0.6825 - val_loss: 0.6259 - val_f1_score: 0.7377
Epoch 7/20
74/74 [==============================] - 38s 512ms/step - loss: 0.5700 - f1_score: 0.7198 - val_loss: 0.5895 - val_f1_score: 0.6499
Epoch 8/20

In [26]:
model.save('../models/ResNet.keras')
#model.save('monReader.keras_tf', save_format='tf')